In [13]:
%matplotlib inline
import pandas as pd
import numpy as np
import json
# plotting
import matplotlib.pyplot as plt
# statistical fitting of models
import statsmodels.api as sm
import statsmodels.formula.api as smf
#opening data
import os
import pathlib
import warnings
#used for plots
from scipy import stats
from mplsoccer import PyPizza, FontManager

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [14]:
#df = pd.read_json("C:/Users/mail4/Desktop/Soccermatics/wyscout/events/events_England.json")
df = pd.read_json("C:/Users/mail4/Desktop/Soccermatics/wyscout/events/events_Germany.json")
df  = df.loc[df.apply (lambda x: len(x.positions) == 2, axis = 1)]
df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],15231,"[{'y': 50, 'x': 50}, {'y': 48, 'x': 50}]",2516739,Pass,2446,1H,2.409746,85,179896442
1,8,Simple pass,[{'id': 1801}],14786,"[{'y': 48, 'x': 50}, {'y': 22, 'x': 22}]",2516739,Pass,2446,1H,2.506082,85,179896443
2,8,Simple pass,[{'id': 1801}],14803,"[{'y': 22, 'x': 22}, {'y': 46, 'x': 6}]",2516739,Pass,2446,1H,6.946706,85,179896444
3,8,Simple pass,[{'id': 1801}],14768,"[{'y': 46, 'x': 6}, {'y': 10, 'x': 20}]",2516739,Pass,2446,1H,10.786491,85,179896445
4,8,Simple pass,[{'id': 1801}],14803,"[{'y': 10, 'x': 20}, {'y': 4, 'x': 27}]",2516739,Pass,2446,1H,12.684514,85,179896446
...,...,...,...,...,...,...,...,...,...,...,...,...
519402,1,Ground attacking duel,"[{'id': 702}, {'id': 1801}]",15317,"[{'y': 9, 'x': 91}, {'y': 6, 'x': 90}]",2517044,Duel,2463,2H,2825.097992,11,251206867
519403,1,Ground defending duel,"[{'id': 703}, {'id': 1801}]",14804,"[{'y': 94, 'x': 10}, {'y': 95, 'x': 0}]",2517044,Duel,2451,2H,2825.519102,12,251206788
519404,1,Ground attacking duel,"[{'id': 701}, {'id': 1802}]",15317,"[{'y': 6, 'x': 90}, {'y': 5, 'x': 100}]",2517044,Duel,2463,2H,2826.053788,11,251206868
519405,8,Simple pass,[{'id': 1802}],14804,"[{'y': 95, 'x': 0}, {'y': 93, 'x': 5}]",2517044,Pass,2451,2H,2828.080925,85,251206790


In [15]:
def wonDuels(df):
    """
    Parameters
    ----------
    df : dataframe
        dataframe with Wyscout event data.

    Returns
    -------
    duels_won: dataframe
        dataframe with percentage of won air and ratio of fouls committed to ground duels won in own half

    """
    #find next event
    next_event = df.shift(-1, fill_value=0)

    #find air duels
    air_duels = df.loc[df["subEventName"] == "Air duel"]
    ad_player = air_duels.groupby(["playerId"]).eventId.count().reset_index()
    ad_player.rename(columns = {"eventId" : "air_duels"}, inplace=True)

    #703 is the id of a won duel
    won_air_duels = air_duels.loc[air_duels.apply (lambda x:{'id':703} in x.tags, axis = 1)]

    #group and sum air duels
    wad_player =  won_air_duels.groupby(["playerId"]).eventId.count().reset_index()
    wad_player.rename(columns = {'eventId':'air_duels_won'}, inplace=True)

    #find ground duels won
    ground_duels = df.loc[df["subEventName"].isin(["Ground defending duel"])]
    gd_player = ground_duels.groupby(["playerId"]).eventId.count().reset_index()
    gd_player.rename(columns = {'eventId' : 'ground_duels'}, inplace=True)

    won_ground_duels = ground_duels.loc[ground_duels.apply (lambda x:{'id':703} in x.tags, axis = 1)]
    wgd_player =  won_ground_duels.groupby(["playerId"]).eventId.count().reset_index()
    wgd_player.rename(columns = {'eventId':'ground_duels_won'}, inplace=True)

    #find won ground duels against dribbles
    vs_dribbles = ground_duels.loc[(df.apply(lambda x :{'id':501} in x.tags, axis=1))|(df.apply(lambda x :{'id':502} in x.tags, axis=1))|(df.apply(lambda x :{'id':503} in x.tags, axis=1))|(df.apply(lambda x :{'id':504} in x.tags, axis=1))]
    vsd_player = vs_dribbles.groupby(["playerId"]).eventId.count().reset_index()
    vsd_player.rename(columns = {'eventId' : 'vs_dribbles'}, inplace=True)

    won_vs_dribbles = vs_dribbles.loc[vs_dribbles.apply (lambda x:{'id':703} in x.tags, axis = 1)]
    wvsd_player = won_vs_dribbles.groupby(["playerId"]).eventId.count().reset_index()
    wvsd_player.rename(columns = {'eventId' : 'won_vs_dribbles'}, inplace=True)

    #find fouls committed
    fouls_committed = df.loc[df["subEventName"].isin(["Foul"])]
    fc_player = fouls_committed.groupby(["playerId"]).eventId.count().reset_index()
    fc_player.rename(columns = {'eventId' : 'fouls_committed'}, inplace=True)

    #find fouls committed and won duels in own half
    won_duels = pd.concat([won_ground_duels,won_air_duels])
    won_duels_own_half = won_duels.loc[won_duels.positions.apply(lambda cell: cell[0]['x'] < 50)]
    wdoh_player = won_duels_own_half.groupby(["playerId"]).eventId.count().reset_index()
    wdoh_player.rename(columns = {'eventId' : 'duels_won_own_half'}, inplace=True)
    fouls_committed_own_half = fouls_committed.loc[fouls_committed.positions.apply(lambda cell: cell[0]['x'] < 50)]
    fcoh_player = fouls_committed_own_half.groupby(["playerId"]).eventId.count().reset_index()
    fcoh_player.rename(columns = {'eventId' : 'fouls_committed_own_half'}, inplace=True)

    #find interceptions - interception tag 1401
    df["playerId"] = next_event["playerId"] # assuming player with next action intercepted the pass
    interceptions = df.loc[(df["eventName"] == "Pass") & (df.apply(lambda x :{'id':1401} in x.tags, axis=1)) & (df.apply(lambda x :{'id':1802} in x.tags, axis=1))]
    int_player = interceptions.groupby(["playerId"]).eventId.count().reset_index()
    int_player.rename(columns = {'eventId' : 'interceptions'}, inplace=True)



    #outer join
    duels_won = wgd_player.merge(gd_player, how = "outer", on = ["playerId"]).merge(fc_player, how = "outer", on = ["playerId"]).merge(ad_player, how = "outer", on = ["playerId"]).merge(wad_player, how = "outer", on = ["playerId"]).merge(wdoh_player, how = "outer", on = ["playerId"]).merge(fcoh_player, how = "outer", on = ["playerId"]).merge(int_player, how = "outer", on = ["playerId"]).merge(vsd_player, how = "outer", on = ["playerId"]).merge(wvsd_player, how = "outer", on = ["playerId"])
    #duels = duels_won.merge(fc_player, how = "outer", on = ["playerId"])
    duels_won = duels_won.fillna(0)

    duels_won["foul_per_won_duel"] = duels_won["fouls_committed"] / (duels_won["ground_duels_won"] + duels_won["air_duels_won"])
    duels_won["air_duels_won%"] = duels_won["air_duels_won"] / duels_won["air_duels"]
    duels_won["ground_duels_won%"] = duels_won["ground_duels_won"] / duels_won["ground_duels"]
    duels_won["foul_per_won_duel_own_half"] = duels_won["fouls_committed_own_half"] / duels_won["duels_won_own_half"]
    duels_won["won_vs_dribbles%"] = duels_won["won_vs_dribbles"] / duels_won["vs_dribbles"]
    #duels_won = duels_won.drop(["ground_duels_won", "fouls_committed", "air_duels_won","fouls_committed_own_half", "duels_won_own_half"], axis = 1)
    return duels_won
#duels = wonDuels(df)
duels = wonDuels(df)
duels

,playerId,ground_duels_won,ground_duels,fouls_committed,air_duels,air_duels_won,duels_won_own_half,fouls_committed_own_half,interceptions,vs_dribbles,won_vs_dribbles,foul_per_won_duel,air_duels_won%,ground_duels_won%,foul_per_won_duel_own_half,won_vs_dribbles%
0,0,1010.0,4739.0,51.0,2202.0,780.0,1131.0,29.0,151.0,1957.0,379.0,0.028492,0.354223,0.213125,0.025641,0.193664
1,77,38.0,150.0,33.0,84.0,39.0,64.0,23.0,14.0,67.0,15.0,0.428571,0.464286,0.253333,0.359375,0.223881
2,91,2.0,14.0,4.0,16.0,7.0,4.0,2.0,1.0,7.0,0.0,0.444444,0.437500,0.142857,0.500000,0.000000
3,110,41.0,144.0,23.0,46.0,18.0,43.0,14.0,7.0,66.0,19.0,0.389831,0.391304,0.284722,0.325581,0.287879
4,159,64.0,219.0,31.0,139.0,72.0,111.0,21.0,15.0,71.0,19.0,0.227941,0.517986,0.292237,0.189189,0.267606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,14822,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,1.000000,NaN,0.000000,NaN
462,15254,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,NaN,NaN,NaN
463,253822,0.0,0.0,0.0,15.0,15.0,14.0,0.0,6.0,0.0,0.0,0.000000,1.000000,NaN,0.000000,NaN
464,337647,0.0,0.0,0.0,3.0,3.0,3.0,0.0,2.0,0.0,0.0,0.000000,1.000000,NaN,0.000000,NaN


In [16]:
#minutes_per_game = pd.read_json("C:/Users/mail4/Desktop/Soccermatics/wyscout/minutes_played/minutes_played_per_game_England.json")
minutes_per_game = pd.read_json("C:/Users/mail4/Desktop/Soccermatics/wyscout/minutes_played/minutes_played_per_game_Germany.json")
minutes = minutes_per_game.groupby(["playerId"]).minutesPlayed.sum().reset_index()

In [17]:
duels=minutes.merge(duels, how='left', on='playerId')
duels

,playerId,minutesPlayed,ground_duels_won,ground_duels,fouls_committed,air_duels,air_duels_won,duels_won_own_half,fouls_committed_own_half,interceptions,vs_dribbles,won_vs_dribbles,foul_per_won_duel,air_duels_won%,ground_duels_won%,foul_per_won_duel_own_half,won_vs_dribbles%
0,77,2326,38.0,150.0,33.0,84.0,39.0,64.0,23.0,14.0,67.0,15.0,0.428571,0.464286,0.253333,0.359375,0.223881
1,91,198,2.0,14.0,4.0,16.0,7.0,4.0,2.0,1.0,7.0,0.0,0.444444,0.437500,0.142857,0.500000,0.000000
2,110,1713,41.0,144.0,23.0,46.0,18.0,43.0,14.0,7.0,66.0,19.0,0.389831,0.391304,0.284722,0.325581,0.287879
3,159,2463,64.0,219.0,31.0,139.0,72.0,111.0,21.0,15.0,71.0,19.0,0.227941,0.517986,0.292237,0.189189,0.267606
4,1722,355,4.0,16.0,4.0,13.0,3.0,4.0,1.0,1.0,6.0,2.0,0.571429,0.230769,0.250000,0.250000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,476054,31,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,NaN,NaN,0.000000,NaN,0.000000
468,491365,892,5.0,52.0,22.0,59.0,12.0,4.0,5.0,7.0,12.0,1.0,1.294118,0.203390,0.096154,1.250000,0.083333
469,493725,10,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,0.000000,NaN,0.000000
470,503185,19,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.000000,NaN,0.333333,NaN,0.333333


In [18]:
duels=duels.fillna(0)
duels=duels.loc[duels["minutesPlayed"]>400]

In [19]:
#loading and filtering players
player_df = pd.read_json("C:/Users/mail4/Desktop/Soccermatics/wyscout/players.json")
defenders = player_df.loc[player_df.apply(lambda x: x.role["name"] == "Defender", axis = 1)]
defenders.rename(columns = {'wyId':'playerId','currentTeamId':'teamId'}, inplace=True)
to_merge = defenders[['playerId', 'shortName', 'teamId']]
duels = duels.merge(to_merge, how = "inner", on = ["playerId"])

In [20]:
duels

,playerId,minutesPlayed,ground_duels_won,ground_duels,fouls_committed,air_duels,air_duels_won,duels_won_own_half,fouls_committed_own_half,interceptions,vs_dribbles,won_vs_dribbles,foul_per_won_duel,air_duels_won%,ground_duels_won%,foul_per_won_duel_own_half,won_vs_dribbles%,shortName,teamId
0,77,2326,38.0,150.0,33.0,84.0,39.0,64.0,23.0,14.0,67.0,15.0,0.428571,0.464286,0.253333,0.359375,0.223881,N. Moisander,2443
1,110,1713,41.0,144.0,23.0,46.0,18.0,43.0,14.0,7.0,66.0,19.0,0.389831,0.391304,0.284722,0.325581,0.287879,J. Willems,2462
2,159,2463,64.0,219.0,31.0,139.0,72.0,111.0,21.0,15.0,71.0,19.0,0.227941,0.517986,0.292237,0.189189,0.267606,J. Gouweleeuw,2481
3,3281,900,14.0,58.0,4.0,29.0,11.0,16.0,2.0,2.0,19.0,4.0,0.160000,0.379310,0.241379,0.125000,0.210526,Juan Bernat,2444
4,3335,847,19.0,85.0,9.0,64.0,43.0,44.0,4.0,7.0,34.0,4.0,0.145161,0.671875,0.223529,0.090909,0.117647,Bartra,684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,366217,1131,20.0,72.0,18.0,80.0,33.0,46.0,10.0,8.0,31.0,7.0,0.339623,0.412500,0.277778,0.217391,0.225806,K. Danso,2481
125,391441,670,10.0,54.0,6.0,17.0,10.0,15.0,2.0,2.0,29.0,4.0,0.300000,0.588235,0.185185,0.133333,0.137931,M. Friedl,2443
126,393247,1267,27.0,79.0,13.0,69.0,39.0,49.0,9.0,5.0,33.0,11.0,0.196970,0.565217,0.341772,0.183673,0.333333,I. Konat\u00e9,2975
127,397028,678,15.0,57.0,13.0,58.0,34.0,40.0,5.0,6.0,19.0,3.0,0.265306,0.586207,0.263158,0.125000,0.157895,D. Zagadou,2447


In [21]:
#loading teams
team_df = pd.read_json("C:/Users/mail4/Desktop/Soccermatics/wyscout/teams.json")
team_df.rename(columns={'wyId':'teamId','name':'teamName'},inplace=True)
team_df = team_df[['teamId','teamName']]
duels = duels.merge(team_df,how="inner", on=["teamId"])

In [22]:
duels

,playerId,minutesPlayed,ground_duels_won,ground_duels,fouls_committed,air_duels,air_duels_won,duels_won_own_half,fouls_committed_own_half,interceptions,vs_dribbles,won_vs_dribbles,foul_per_won_duel,air_duels_won%,ground_duels_won%,foul_per_won_duel_own_half,won_vs_dribbles%,shortName,teamId,teamName
0,77,2326,38.0,150.0,33.0,84.0,39.0,64.0,23.0,14.0,67.0,15.0,0.428571,0.464286,0.253333,0.359375,0.223881,N. Moisander,2443,Werder Bremen
1,11095,2749,50.0,208.0,21.0,128.0,71.0,100.0,14.0,7.0,79.0,17.0,0.173554,0.554688,0.240385,0.140000,0.215190,M. Veljkovic,2443,Werder Bremen
2,15771,1427,23.0,98.0,14.0,68.0,49.0,53.0,7.0,9.0,48.0,9.0,0.194444,0.720588,0.234694,0.132075,0.187500,S. Langkamp,2443,Werder Bremen
3,51217,2615,40.0,174.0,26.0,100.0,47.0,53.0,8.0,14.0,72.0,15.0,0.298851,0.470000,0.229885,0.150943,0.208333,L. Augustinsson,2443,Werder Bremen
4,82340,2923,46.0,253.0,41.0,229.0,110.0,89.0,20.0,14.0,86.0,13.0,0.262821,0.480349,0.181818,0.224719,0.151163,T. Gebre Selassie,2443,Werder Bremen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,216074,3163,47.0,215.0,23.0,49.0,13.0,50.0,11.0,15.0,96.0,18.0,0.383333,0.265306,0.218605,0.220000,0.187500,C. G\u00fcnter,2453,Freiburg
120,255732,888,9.0,48.0,9.0,31.0,15.0,18.0,7.0,2.0,21.0,5.0,0.375000,0.483871,0.187500,0.388889,0.238095,P. Lienhart,2453,Freiburg
121,272378,2009,41.0,181.0,22.0,49.0,24.0,48.0,11.0,6.0,95.0,21.0,0.338462,0.489796,0.226519,0.229167,0.221053,P. Stenzel,2453,Freiburg
122,336829,1997,32.0,121.0,30.0,140.0,83.0,84.0,20.0,14.0,41.0,7.0,0.260870,0.592857,0.264463,0.238095,0.170732,R. Koch,2453,Freiburg


In [23]:
possesion_dict = {}
#for every row in the dataframe
for i, row in minutes_per_game.iterrows():
    #take player id, team id and match id, minute in and minute out
    player_id, team_id, match_id = row["playerId"], row["teamId"], row["matchId"]
    #create a key in dictionary if player encounterd first time
    if not str(player_id) in possesion_dict.keys():
        possesion_dict[str(player_id)] = {'team_passes': 0, 'all_passes' : 0}
    min_in = row["player_in_min"]*60
    min_out = row["player_out_min"]*60

    #get the dataframe of events from the game
    match_df = df.loc[df["matchId"] == match_id].copy()
    #add to 2H the highest value of 1H
    match_df.loc[match_df["matchPeriod"] == "2H", 'eventSec'] = match_df.loc[match_df["matchPeriod"] == "2H", 'eventSec'] + match_df.loc[match_df["matchPeriod"] == "1H"]["eventSec"].iloc[-1]
    #take all events from this game and this period
    player_in_match_df = match_df.loc[match_df["eventSec"] > min_in].loc[match_df["eventSec"] <= min_out]
    #take all passes and won duels as described
    all_passes = player_in_match_df.loc[player_in_match_df["eventName"].isin(["Pass", "Duel"])]
    #adjusting for no passes in this period (Tuanzebe)
    if len(all_passes) > 0:
        #removing lost air duels
        no_contact = all_passes.loc[all_passes["subEventName"].isin(["Air duel", "Ground defending duel","Ground loose ball duel"])].loc[all_passes.apply(lambda x:{'id':701} in x.tags, axis = 1)]
        all_passes = all_passes.drop(no_contact.index)
    #take team passes
    team_passes = all_passes.loc[all_passes["teamId"] == team_id]
    #append it {player id: {team passes: sum, all passes : sum}}
    possesion_dict[str(player_id)]["team_passes"] += len(team_passes)
    possesion_dict[str(player_id)]["all_passes"] += len(all_passes)

#calculate possesion for each player
percentage_dict = {key: value["team_passes"]/value["all_passes"] if value["all_passes"] > 0 else 0 for key, value in possesion_dict.items()}
#create a dataframe
percentage_df = pd.DataFrame(percentage_dict.items(), columns = ["playerId", "possesion"])
percentage_df["playerId"] = percentage_df["playerId"].astype(int)
duels = duels.merge(percentage_df, how = "left", on = ["playerId"])

In [24]:
duels

,playerId,minutesPlayed,ground_duels_won,ground_duels,fouls_committed,air_duels,air_duels_won,duels_won_own_half,fouls_committed_own_half,interceptions,...,won_vs_dribbles,foul_per_won_duel,air_duels_won%,ground_duels_won%,foul_per_won_duel_own_half,won_vs_dribbles%,shortName,teamId,teamName,possesion
0,77,2326,38.0,150.0,33.0,84.0,39.0,64.0,23.0,14.0,...,15.0,0.428571,0.464286,0.253333,0.359375,0.223881,N. Moisander,2443,Werder Bremen,0.486420
1,11095,2749,50.0,208.0,21.0,128.0,71.0,100.0,14.0,7.0,...,17.0,0.173554,0.554688,0.240385,0.140000,0.215190,M. Veljkovic,2443,Werder Bremen,0.463976
2,15771,1427,23.0,98.0,14.0,68.0,49.0,53.0,7.0,9.0,...,9.0,0.194444,0.720588,0.234694,0.132075,0.187500,S. Langkamp,2443,Werder Bremen,0.481168
3,51217,2615,40.0,174.0,26.0,100.0,47.0,53.0,8.0,14.0,...,15.0,0.298851,0.470000,0.229885,0.150943,0.208333,L. Augustinsson,2443,Werder Bremen,0.471881
4,82340,2923,46.0,253.0,41.0,229.0,110.0,89.0,20.0,14.0,...,13.0,0.262821,0.480349,0.181818,0.224719,0.151163,T. Gebre Selassie,2443,Werder Bremen,0.474609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,216074,3163,47.0,215.0,23.0,49.0,13.0,50.0,11.0,15.0,...,18.0,0.383333,0.265306,0.218605,0.220000,0.187500,C. G\u00fcnter,2453,Freiburg,0.446702
120,255732,888,9.0,48.0,9.0,31.0,15.0,18.0,7.0,2.0,...,5.0,0.375000,0.483871,0.187500,0.388889,0.238095,P. Lienhart,2453,Freiburg,0.413881
121,272378,2009,41.0,181.0,22.0,49.0,24.0,48.0,11.0,6.0,...,21.0,0.338462,0.489796,0.226519,0.229167,0.221053,P. Stenzel,2453,Freiburg,0.466228
122,336829,1997,32.0,121.0,30.0,140.0,83.0,84.0,20.0,14.0,...,7.0,0.260870,0.592857,0.264463,0.238095,0.170732,R. Koch,2453,Freiburg,0.455498


In [25]:
duels.rename(columns={'possesion':'possession'}, inplace=True)
duels["possession_opp"] = 1-duels["possession"]
duels

,playerId,minutesPlayed,ground_duels_won,ground_duels,fouls_committed,air_duels,air_duels_won,duels_won_own_half,fouls_committed_own_half,interceptions,...,foul_per_won_duel,air_duels_won%,ground_duels_won%,foul_per_won_duel_own_half,won_vs_dribbles%,shortName,teamId,teamName,possession,possession_opp
0,77,2326,38.0,150.0,33.0,84.0,39.0,64.0,23.0,14.0,...,0.428571,0.464286,0.253333,0.359375,0.223881,N. Moisander,2443,Werder Bremen,0.486420,0.513580
1,11095,2749,50.0,208.0,21.0,128.0,71.0,100.0,14.0,7.0,...,0.173554,0.554688,0.240385,0.140000,0.215190,M. Veljkovic,2443,Werder Bremen,0.463976,0.536024
2,15771,1427,23.0,98.0,14.0,68.0,49.0,53.0,7.0,9.0,...,0.194444,0.720588,0.234694,0.132075,0.187500,S. Langkamp,2443,Werder Bremen,0.481168,0.518832
3,51217,2615,40.0,174.0,26.0,100.0,47.0,53.0,8.0,14.0,...,0.298851,0.470000,0.229885,0.150943,0.208333,L. Augustinsson,2443,Werder Bremen,0.471881,0.528119
4,82340,2923,46.0,253.0,41.0,229.0,110.0,89.0,20.0,14.0,...,0.262821,0.480349,0.181818,0.224719,0.151163,T. Gebre Selassie,2443,Werder Bremen,0.474609,0.525391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,216074,3163,47.0,215.0,23.0,49.0,13.0,50.0,11.0,15.0,...,0.383333,0.265306,0.218605,0.220000,0.187500,C. G\u00fcnter,2453,Freiburg,0.446702,0.553298
120,255732,888,9.0,48.0,9.0,31.0,15.0,18.0,7.0,2.0,...,0.375000,0.483871,0.187500,0.388889,0.238095,P. Lienhart,2453,Freiburg,0.413881,0.586119
121,272378,2009,41.0,181.0,22.0,49.0,24.0,48.0,11.0,6.0,...,0.338462,0.489796,0.226519,0.229167,0.221053,P. Stenzel,2453,Freiburg,0.466228,0.533772
122,336829,1997,32.0,121.0,30.0,140.0,83.0,84.0,20.0,14.0,...,0.260870,0.592857,0.264463,0.238095,0.170732,R. Koch,2453,Freiburg,0.455498,0.544502


In [26]:
#calculate value adjusted
for column in duels.columns[2:12]:
    duels[column + "_adjusted_per90"] = duels.apply(lambda x: (x[column]/x["possession_opp"])*90/x["minutesPlayed"], axis = 1)

In [27]:
duels

,playerId,minutesPlayed,ground_duels_won,ground_duels,fouls_committed,air_duels,air_duels_won,duels_won_own_half,fouls_committed_own_half,interceptions,...,ground_duels_won_adjusted_per90,ground_duels_adjusted_per90,fouls_committed_adjusted_per90,air_duels_adjusted_per90,air_duels_won_adjusted_per90,duels_won_own_half_adjusted_per90,fouls_committed_own_half_adjusted_per90,interceptions_adjusted_per90,vs_dribbles_adjusted_per90,won_vs_dribbles_adjusted_per90
0,77,2326,38.0,150.0,33.0,84.0,39.0,64.0,23.0,14.0,...,2.862912,11.300969,2.486213,6.328543,2.938252,4.821747,1.732815,1.054757,5.047766,1.130097
1,11095,2749,50.0,208.0,21.0,128.0,71.0,100.0,14.0,7.0,...,3.053889,12.704180,1.282634,7.817957,4.336523,6.107779,0.855089,0.427545,4.825145,1.038322
2,15771,1427,23.0,98.0,14.0,68.0,49.0,53.0,7.0,9.0,...,2.795890,11.912921,1.701846,8.266108,5.956460,6.442702,0.850923,1.094044,5.834900,1.094044
3,51217,2615,40.0,174.0,26.0,100.0,47.0,53.0,8.0,14.0,...,2.606748,11.339354,1.694386,6.516870,3.062929,3.453941,0.521350,0.912362,4.692146,0.977530
4,82340,2923,46.0,253.0,41.0,229.0,110.0,89.0,20.0,14.0,...,2.695810,14.826952,2.402787,13.420443,6.446501,5.215805,1.172091,0.820464,5.039992,0.761859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,216074,3163,47.0,215.0,23.0,49.0,13.0,50.0,11.0,15.0,...,2.417032,11.056634,1.182803,2.519884,0.668541,2.571310,0.565688,0.771393,4.936916,0.925672
120,255732,888,9.0,48.0,9.0,31.0,15.0,18.0,7.0,2.0,...,1.556275,8.300132,1.556275,5.360502,2.593791,3.112549,1.210436,0.345839,3.631308,0.864597
121,272378,2009,41.0,181.0,22.0,49.0,24.0,48.0,11.0,6.0,...,3.441047,15.190964,1.846415,4.112471,2.014271,4.028543,0.923208,0.503568,7.973158,1.762488
122,336829,1997,32.0,121.0,30.0,140.0,83.0,84.0,20.0,14.0,...,2.648590,10.014982,2.483053,11.587583,6.869781,6.952550,1.655369,1.158758,3.393506,0.579379


In [28]:
#duels.to_csv("england_defender_radardata.csv")
duels.to_csv("germany_defender_radardata.csv")